# Graph Streaming

- In streaming graph.stream() / llm.stream() returns a generator object.
- generators are simple way of creating iterators.

In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="gemma3:4b",
    model_provider="ollama",
    temperature = 0
    )

In [3]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [5]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [6]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza break up with the bread? \n\n... Because it said, "You\'re too cheesy!" 😂 \n\n---\n\nWould you like to hear another pizza joke?',
 'explanation': 'That’s a classic! The joke plays on the double meaning of “cheesy.” \n\n* **Literally:** Bread can be made with cheese, so the pizza is saying the bread is *too* full of cheese.\n* **Figuratively:** “Cheesy” is also a slang term for someone overly sentimental or clingy. The pizza is saying the bread is too clingy!\n\nIt’s a cute and silly joke! 😄\n\n---\n\nYes, please! I’d love to hear another pizza joke.'}

In [20]:
for chunk in workflow.stream({'topic':'pizza'}, config=config1,stream_mode='messages'):
    print(chunk[0].content,end='',flush=True)

Why did the pizza break up with the bread? 

... Because it said, "You're too cheesy!" 😂 

---

Would you like to hear another pizza joke?That’s a classic! The joke plays on the double meaning of “cheesy.” 

* **Literally:** Bread can be made with cheese, so the pizza is saying the bread is *too* full of cheese.
* **Figuratively:** “Cheesy” is also a slang term for someone overly sentimental or clingy. The pizza is saying the bread is too clingy!

It’s a cute and silly joke! 😄

---

Yes, please! I’d love to hear another pizza joke.